In [ ]:
from google.colab import drive

drive.mount("gdrive")

Mounted at gdrive


## Caption model

In [ ]:
!pip install diffusers -U -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:00


In [ ]:
from tqdm import tqdm

from transformers import pipeline
from diffusers.utils import load_image


class Captioner:
    def __init__(self,
                 pipeline_name: str = "image-to-text",
                 model_name: str = "Salesforce/blip-image-captioning-base",
                 base_prompt: str = "The main subject of this picture is a"):

        self.base_prompt = base_prompt
        self.caption_model = pipeline(
            pipeline_name,
            model=model_name,
            prompt=base_prompt
        )

    def get_captions(self,
                     images: list,
                     max_new_tokens: int = 20):
        captions = self.caption_model(images, max_new_tokens=max_new_tokens)
        dataset = [
            {
                "image": file_name,
                "text": caption[0]["generated_text"].lower().replace(self.base_prompt.lower(), "").strip()
            }
            for file_name, caption in tqdm(zip(images, captions), desc="Dataset processing...")
        ]

        return dataset


caption = Captioner(model_name="Salesforce/blip-image-captioning-large")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

In [ ]:
%cd /content/gdrive/MyDrive/aitalenthack/unsplash_images


/content/gdrive/MyDrive/aitalenthack/unsplash_images


In [ ]:
import json

images_with_caption = []
with open("metadata.jsonl", "r") as file:
    for line in file:
        images_with_caption.append(json.loads(line))

file_names_captioned = [data["file_name"] for data in images_with_caption]


In [ ]:
import os
import json

images = os.listdir("./")
images = [image for image in images if (image not in file_names_captioned)
          and (image.endswith(".jpeg"))]


In [ ]:
import pickle
from tqdm import tqdm

batch_size = 8
datasets = []

for idx in tqdm(range(0, len(images), batch_size)):
    dataset = caption.get_captions(images[idx:idx+batch_size])
    datasets.extend(dataset)


  0%|          | 0/8 [00:00<?, ?it/s]


Dataset processing...: 8it [00:00, 47393.27it/s]
 12%|█▎        | 1/8 [02:10<15:12, 130.37s/it]


Dataset processing...: 8it [00:00, 54120.05it/s]
 25%|██▌       | 2/8 [04:07<12:13, 122.30s/it]


Dataset processing...: 8it [00:00, 56679.78it/s]
 38%|███▊      | 3/8 [05:57<09:45, 117.05s/it]


Dataset processing...: 8it [00:00, 47662.55it/s]
 50%|█████     | 4/8 [07:54<07:47, 116.75s/it]


Dataset processing...: 8it [00:00, 50382.03it/s]
 62%|██████▎   | 5/8 [09:52<05:51, 117.17s/it]


Dataset processing...: 8it [00:00, 49056.19it/s]
 75%|███████▌  | 6/8 [11:54<03:57, 118.97s/it]


Dataset processing...: 8it [00:00, 51306.47it/s]
 88%|████████▊ | 7/8 [13:46<01:56, 116.58s/it]


Dataset processing...: 8it [00:00, 51622.20it/s]
100%|██████████| 8/8 [15:50<00:00, 118.85s/it]


In [ ]:
import pickle

with open("interier_second_part.pickle", "wb") as handle:
    pickle.dump(datasets, handle)
